In [1]:
import sqlite3
import requests as r
from  bs4 import BeautifulSoup
from lib.toolbox import gen_header
import time 
import random
import json

header_str = 'User-Agent:Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'
HEADER = gen_header(header_str)
carName_list = ['PORSCHE','NISSAN','TOYOTA','MITSUBISHI','HONDA','FORD','AUDI','MERCEDES BENZ','BMW','LEXUS','VOLKSWAGEN','MAZDA','SUBARU','SUZUKI','VOLVO']

In [2]:
#Create db
def creatDB():   
    db = sqlite3.connect("./Mobile01.db")
    cur = db.cursor()
    cur.execute("drop table Mobile01")
    cur.execute("create table Mobile01(url text,title text,author text,tm int,Board text,content text,reply_no int,replies text);")
    db.commit()
creatDB()

In [3]:
#pages of innerURL
def contentPage(url):      
    res = r.get(url,headers=HEADER)
    soup = BeautifulSoup(res.text,'lxml')
    contentTag = soup.select(".pagination > a")
    tagLen = len(contentTag)
#     print(tagLen)
    
    if tagLen < 8:
        contentPage = int(contentTag[3].text)
    else :
        contentPage = int(contentTag[tagLen-1].text)
    return contentPage

In [4]:
#num of totalArticle
def reply_no(url):
    contentURL = url+"&p="+str(contentPage(url))
#     print(contentPage(url))
    reply_res = r.get(contentURL,headers=HEADER)
    soup = BeautifulSoup(reply_res.text,'lxml')
    reply_no = int(len(soup.select('main > article')))
#     print(reply_no)
    reply_id = int(soup.select('.date')[reply_no-1].text.split('\xa0')[2][1:])
    return reply_id

In [5]:
#content od innerURL
def getSubContent(url):
    print(url)
    replies = []
    for page in range(1,contentPage(url)+1):    #此評論的所有的內頁
        print(page)
        contentURL = url+"&p="+str(page)
        print(contentURL)
        time.sleep(int(random.random()*3+1))
        reply_res = r.get(contentURL,headers=HEADER)
        repli_soup = BeautifulSoup(reply_res.text,'lxml')
        reply_no = int(len(repli_soup.select('main > article')))
        print("article_no : ",reply_no)
        while (reply_no > 0):
            reply = {}
            reply_id = int(repli_soup.select('.date')[reply_no-1].text.split('\xa0')[2][1:])
#             print(reply_id)
            if reply_id != 1:
                author_id = repli_soup.select('.fn > a')[reply_no-1].text
#                 print("author_id",author_id)

                reply_time = repli_soup.select('.date')[reply_no-1].text.split('\xa0')[0]
                poTimeArray = time.strptime(reply_time, "%Y-%m-%d %H:%M")
                tm = int(time.mktime(poTimeArray))
#                 print("tm",tm)

                content = repli_soup.select('.single-post-content')[reply_no-1]
                for blockquote in content.find_all('blockquote'):
                    blockquote.decompose()
                print("content : \n",content.text.strip())

                reply["reply_id"] = reply_id
                reply["tm"] = tm
#                 reply["content"] = content
                reply["author_id"] = str(author_id)
                replies.append(reply)
            reply_no -= 1
    replies =  json.dumps(replies)

    return replies
# getSubContent("https://www.mobile01.com/topicdetail.php?f=346&t=2574949")

In [ ]:
def main(url):
    conn = sqlite3.connect('./Mobile01.db')
    cursor = conn.cursor()

    res = r.get(url,headers=HEADER)
    soup = BeautifulSoup(res.text,'lxml')

    title = soup.select_one('.topic').text
    print("title : ",title)

    author = soup.select_one('.fn > a').text
    print("author : ",author)

    poTime = soup.select_one('.date').text.split('\xa0')[0]
    poTimeArray = time.strptime(poTime, "%Y-%m-%d %H:%M")
    tm = int(time.mktime(poTimeArray))
    print("tm : ",tm)

    boardTag = soup.select('.nav > a')
    Board = boardTag[3].text
    print("Board : ",Board)

    content = soup.select_one('.single-post-content').text
    print("content : ",content)

    reply = reply_no(url)
    print("reply_no : "+str(reply_no(url)))
    
    replies = getSubContent(url)
    print("replies : ",replies)
    try:
        cursor.execute("INSERT into Mobile01 VALUES(?,?,?,?,?,?,?,?)",
                        (url, title, author, tm, Board, content, reply_no, replies))
        conn.commit()
    except Exception as e:
        print(e)
        conn.rollback()    


In [ ]:
#take url from csv to function 

#for carName in carName_list:
with open('./project/{}.csv'.format('AUDI'), 'r') as fr:
    urls = fr.read().strip().split()
    for url in urls:
        print(url)
        print(contentPage(url))
        time.sleep(int(random.random()*3+1))
        main(url)

https://www.mobile01.com/topicdetail.php?f=606&t=4940549
11
title :  請問現在A6 35tfsi Avant折扣
author :  bigeyes1205
tm :  1475320560
Board :  Audi
content :  
最近有想考慮換這台旅行車，不知道版上有沒有可以介紹比較優的菜單，最好是台中的，有看過版上討論可以折價53萬？不知道真的還假的？目前是開B6 1.8t avant 老了該換車了


reply_no : 101
https://www.mobile01.com/topicdetail.php?f=606&t=4940549


In [ ]:
conn = sqlite3.connect('./Mobile01.db')
# cursor = conn.cursor()
rows = conn.execute("SELECT * FROM Mobile01")
for row in rows:
    print(row)
conn.close()
# def check_duplicates(content_dict):
#     conn = sqlite3.connect('/home/ubuntu/python/III/Python/final_project/Crawler/icars.db')
#     cursor = conn.cursor()

#     title = content_dict['標題']
#     color = content_dict['色系']
#     cc = content_dict['排氣量']
#     equip = content_dict['配備']
#     location = content_dict['所在地']

#     num = list(
#         cursor.execute('SELECT * FROM icars WHERE title = ? AND cc = ? AND color = ?  AND equip = ? AND location = ?',
#                        (title, cc, color, equip, location)))
#     if num != 0:
#         return False
#     else:
#         return True


In [1]:
import sqlite3
import requests as r

res = r.get(url,headers=HEADER)
soup = BeautifulSoup(res.text,'lxml')

NameError: name 'url' is not defined